In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator

from os.path import exists

import gdown
import zipfile

from datetime import datetime

### Data Loading

#### Data Frame Columns

 `current_out`: current withdrawn from the box by the consumer <br>
 `current_in`: current supplied to the box from the solar panel <br>
 `current`: `current_out` - `current-in` --> net current that goes in or out of the battery <br>
 `temperature`: temperature in deg C <br>
 `timestamp`: YYYY-MM-DD HH:MM:SS <br>
 `panel_voltage`: voltage of the battery in V <br>
 `state_of_charge_percent`: state of charge of the battery in percent <br>
 `battery_id`: unique identifier of the battery <br>

In [2]:
# Load times series csv file
# Each file contains data from approximately 50 devices
ts1_df = pd.read_csv('..\data/devices1.csv')
ts2_df = pd.read_csv('..\data/devices2.csv')
ts3_df = pd.read_csv('..\data/devices3.csv')
ts4_df = pd.read_csv('..\data/devices4.csv')
 # Concatenate into one big dataframe
ts_df = pd.concat([ts1_df, ts2_df, ts3_df, ts4_df], axis = 0, ignore_index = True)

Downsample the ts_df dataframe by removing rows that contain NaN values.

In [3]:
ts_df_lean = ts_df.dropna()

Add power column and abs(power) column

In [4]:
ts_df_lean

,battery_voltage,current,current_in,current_out,panel_voltage,state_of_charge_percent,temperature,timestamp,usb_current,voltage,battery_id
4,7.075,-2.276,2.278,0.002,7.350,100.0,28.287938,2021-03-02 06:41:24,0.002,7.280,0
9,6.720,0.000,0.002,0.002,0.005,102.0,27.152667,2021-03-02 06:51:24,0.002,6.735,0
14,6.710,0.010,0.002,0.012,0.005,102.0,26.919203,2021-03-02 07:01:24,0.002,6.725,0
19,6.705,0.010,0.002,0.012,0.005,102.0,27.003891,2021-03-02 07:11:24,0.002,6.720,0
24,6.705,0.010,0.002,0.012,0.005,102.0,27.017624,2021-03-02 07:21:24,0.002,6.715,0
...,...,...,...,...,...,...,...,...,...,...,...
20890488,6.715,-0.824,0.832,0.008,6.760,34.0,27.251087,2021-07-05 06:55:38,0.002,6.780,199
20890493,6.725,-0.906,0.914,0.008,6.780,35.0,27.386130,2021-07-05 07:05:38,0.002,6.795,199
20890496,6.730,-0.954,0.962,0.008,6.790,36.0,27.484550,2021-07-05 07:11:35,0.000,6.800,199
20890501,6.740,-1.030,1.038,0.008,6.805,38.0,27.594415,2021-07-05 07:21:36,0.002,6.810,199


In [5]:
ts_df_lean["power"] = ts_df_lean["battery_voltage"]*ts_df_lean["current"]
ts_df_lean["abs(power)"] = ts_df_lean["power"].abs()

Convert timestamp to datetime format and add columns for 'year' and 'month'

In [6]:
# convert timestamp column to datetime format. It has useful functionality, such as getting the 'month' or 'year'
ts_df_lean["timestamp"] = pd.to_datetime(ts_df_lean["timestamp"])

In [7]:
#Add year, and month as dataframe columns
year = np.empty((len(ts_df_lean)), dtype=int)
month = np.empty((len(ts_df_lean)), dtype=int)
for i in range(len(ts_df_lean)):
    year[i] = ts_df_lean["timestamp"].iloc[i].year
    month[i] = ts_df_lean["timestamp"].iloc[i].month

In [8]:
ts_df_lean["year"] = year
ts_df_lean["month"] = month

Create two new time-related columns and add them to the dataframe.<br>
Column 'time_battery_sec' contains values which start at 0 for every battery, subsequently counting the operating time of the battery in seconds. <br>
Column 'dt_sec' gives the sampling time step. Will be useful later to calculate time-weighted averages. At the first row of every battery, dt is zero.

In [9]:
# Had to make some conversions back to string to get this to work.
# Some complications were due to numpy changing the format of datetime objects before storing them.
time_battery_sec = np.empty((0,1))
dt_sec = np.empty((0,1))
for battery in range(0,200):
    single_battery_df = ts_df_lean.loc[ts_df_lean['battery_id']==battery]
    # Get the starting timestamp of this battery. This will be time zero.
    # Also place in this more convenient datetime format which will allow subtraction of datetimes.
    datetime_0 = datetime.strptime(str(single_battery_df["timestamp"].iloc[0]), "%Y-%m-%d %H:%M:%S")
    # Initialise numpy array to store battery_time. Loops using pandas would be too slow.
    battery_time = single_battery_df["timestamp"].astype(str).to_numpy()
    for i, entry in enumerate(battery_time):
        datetime_1 = datetime.strptime(str(entry), "%Y-%m-%d %H:%M:%S")
        battery_time[i] = float((datetime_1-datetime_0).total_seconds())
    # Convert to float:
    battery_time = battery_time.astype(float)
    battery_dt = np.zeros((battery_time.shape[0],)) #Useful mostly because it puts a zero at the -1 index for this battery_id.
    battery_dt[0:-1] = battery_time[1:] - battery_time[0:-1]
    time_battery_sec = np.append(time_battery_sec, battery_time)
    dt_sec = np.append(dt_sec, battery_dt)

In [10]:
#Sanity check of dimensions:
print('ts_df_lean.shape: ', ts_df_lean.shape)
print('time_battery_seconds.shape: ', time_battery_sec.shape)
print('dt.shape: ', dt_sec.shape)

ts_df_lean.shape:  (4505554, 15)
time_battery_seconds.shape:  (4505554,)
dt.shape:  (4505554,)


In [11]:
# Add new pandas Series to ts_df_lean
ts_df_lean["time_battery_sec"] = time_battery_sec
ts_df_lean["dt_sec"] = dt_sec

In [12]:
%store ts_df_lean

Stored 'ts_df_lean' (DataFrame)
